In [1]:
import requests
import sys
import os
import re

In [2]:
def download(url, filename):
    name = re.search('[0-9a-z+_-]+\.[0-9a-z]+$', filename).group(0)
    
    response = requests.get(url, stream=True)
    total = response.headers.get('content-length')
    
    if os.path.exists(filename):
        local = os.path.getsize(filename)
        
        if total is None:
            #response.content
            fail = '~ url fail\n'
            sys.stdout.write('\r[{}{}] {}'.format(' ', '', f'{name} {fail}'))
            sys.stdout.flush()
        
            return
        if int(total) == local:
            sys.stdout.write(f'[*] {name} - already\n')
            sys.stdout.flush()
            
            return
        else:
            fail = '~ checksum fail'
            sys.stdout.write('\r[{}{}] {}'.format(' ', '', f'{name} {fail}'))
            sys.stdout.flush()
            
    with open(filename, 'wb') as f:
        downloaded = 0
        total = int(total)
        sys.stdout.write('\r[{}{}] {}'.format(' ', '', ' ' * 1000))
        sys.stdout.flush()
        load = '|'
        
        for data in response.iter_content(chunk_size=max(int(total/1000), 1024*1024)):
            downloaded += len(data)
            f.write(data)
            done = int(13*downloaded/total)
            
            if load == '|': load = '/';
            elif load == '/': load = '-';
            elif load == '-': load = '\\';
            elif load == '\\': load = '|';
            
            sys.stdout.write('\r[{}{}] {}'.format('#' * done + load, '.' * (12-done), f'{name} ~ {round(downloaded/1048576)}Mb'))
            sys.stdout.flush()
        
        sys.stdout.write('\r[{}{}] {}'.format('*', '', ' ' * 1000))
        sys.stdout.write('\r[{}{}] {}'.format('*', '', f'{name} - done\n'))
        sys.stdout.flush()


In [3]:
#создание папок
if os.path.exists('./datasets/ds_bert') == False: os.makedirs('./datasets/ds_bert', exist_ok=True);
if os.path.exists('./subsets/toxic_comments') == False: os.makedirs('./subsets/toxic_comments', exist_ok=True);

#часть ссылок для локального дублирования
urls = ['datasets/ds_bert/vocab.txt',
        'datasets/ds_bert/bert_config.json',
        'datasets/ds_bert/rubert_model.bin',
        'datasets/toxic_comments.csv',
        'subsets/toxic_comments/bert.csv',
        'subsets/toxic_comments/f1_choice.csv']

#цикл функцией по ссылкам
[download(f'http://sample.ru/{url}', f'./{url}') for url in urls]

#без цикла - откуда - куда
download('http://sample.ru/subsets/toxic_comments/f1_selected.csv', './subsets/toxic_comments/f1_selected.csv')
pass

[ ] vocab.txt ~ url fail
[ ] bert_config.json ~ url fail
[ ] rubert_model.bin ~ url fail
[ ] toxic_comments.csv ~ url fail
[ ] bert.csv ~ url fail
[ ] f1_choice.csv ~ url fail
[ ] f1_selected.csv ~ url fail
